In [4]:
#Montamos el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Leemos Los programas .asm -> Tener cuidado con las rutas
def cargarProgramas():
  import requests
  with open("/content/drive/MyDrive/Arquitectura/Practica4/Add.asm") as program:
    add = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/Max.asm") as program:
    max = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/MaxL.asm") as program:
    maxL = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/Pong.asm") as program:
    pong = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/PongL.asm") as program:
    pongL = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/Rect.asm") as program:
    rect = program.read()
  with open("/content/drive/MyDrive/Arquitectura/Practica4/RectL.asm") as program:
    rectL = program.read()
  return add, max, maxL, pong, pongL, rect, rectL

In [7]:
add, max, maxL, pong, pongL, rect, rectL = cargarProgramas()

In [8]:
#Creamos la funcion para eliminar los comentarios del codigo en .asm:
def del_com_espacios(program):
    array_lines = program.split("\n")
    cleaned_lines = []
    for line in array_lines:
        # Eliminar comentarios
        line_without_comment = line.split("//")[0].rstrip()
        # Eliminar espacios en blanco al principio y al final de la línea
        cleaned_line = line_without_comment.lstrip().rstrip()
        if cleaned_line != "":
            cleaned_lines.append(cleaned_line)
    return "\n".join(cleaned_lines)

# Parseador

In [9]:
class parseo():
  def __init__(self, linea): #Constructor
    self.linea = linea
    self.tipo = None
    self.asignarTipo()

  def asignarTipo(self):
    if self.linea.startswith('@'):
      self.tipo = 'A'
    elif self.linea.startswith('('):
      self.tipo = 'L'
    else:
      self.tipo = 'C'

  def get_valor(self):
    if(self.tipo != 'A'):
            return None
    return self.linea[1:].split(' ')[0]

  def get_destino(self):
    indice = self.linea.find('=')
    if(self.tipo != 'C' or indice == -1):
      return None
    return self.linea[:indice]

  def get_operacion(self):
    indice_igual = self.linea.find("=")
    indice_punto_coma = self.linea.find(";")
    if(self.tipo != 'C'):
      return None
    if(indice_igual != -1 and indice_punto_coma != -1):
      return self.linea[indice_igual+1:indice_punto_coma]
    elif(indice_igual == -1 and indice_punto_coma != -1 ):
      return self.linea[:indice_punto_coma]
    elif(indice_igual != -1 and indice_punto_coma == -1):
      return self.linea[indice_igual+1:]
    elif(indice_igual == -1 and indice_punto_coma == -1):
      return self.linea

  def get_salto(self):
    indice_punto_coma = self.linea.find(";")
    if(self.tipo != 'C' or indice_punto_coma == -1):
        return None
    return self.linea[indice_punto_coma+1:]

  def get_etiqueta(self):
    indice_1 = self.linea.find("(")
    indice_2 = self.linea.find(")")
    if(self.tipo != "L"):
      return None
    if(indice_1 != -1 and indice_2 != -1):
      return self.linea[indice_1+1:indice_2]
    return None


# Codificador

In [10]:
def codigo_destino(destino):
  diccionario = {
      None : "000",
      "M"  : "001",
      "D"  : "010",
      "MD" : "011",
      "A"  : "100",
      "AM" : "101",
      "AD" : "110",
      "ADM": "111"
      }
  return diccionario[destino]

def codigo_operacion(operacion):
  diccionario = {
      None : None,
      "0"  :  "0101010",
      "1"  :  "0111111",
      "-1" :  "0111010",
      "D"  :  "0001100",
      "A"  :  "0110000",
      "M"  :  "1110000",
      "!D" :  "0001101",
      "!A" :  "0110001",
      "!M" :  "1110001",
      "-D" :  "0001111",
      "-A" :  "0110011",
      "-M" :  "1110011",
      "D+1":  "0011111",
      "A+1":  "0110111",
      "M+1":  "1110111",
      "D-1":  "0001110",
      "A-1":  "0110010",
      "M-1":  "1110010",
      "D+A":  "0000010",
      "D+M":  "1000010",
      "D-A":  "0010011",
      "D-M":  "1010011",
      "A-D":  "0000111",
      "M-D":  "1000111",
      "D&A":  "0000000",
      "D&M":  "1000000",
      "D|A":  "0010101",
      "D|M":  "1010101"
    }
  return diccionario[operacion]

def codigo_salto(salto):
  diccionario = {
      None  : "000",
      "JGT" : "001",
      "JEQ" : "010",
      "JGE" : "011",
      "JLT" : "100",
      "JNE" : "101",
      "JLE" : "110",
      "JMP" : "111"
  }
  return diccionario[salto]

def codigo_valor(valor, diccionario):
    binario = ""
    valor = valor
    if valor.isdigit():
        binario =  bin(int(valor))[2:]
    elif valor in diccionario:
        binario =  bin(diccionario[valor])[2:]
    while(len(binario) != 15):
        binario = "0"+ binario
    return binario

def agregar_al_diccionario(diccionario, lineas):
  contador = 0;
  for linea in lineas:
    if(linea.tipo != 'L'):
      contador = contador +1
    else:
      etiqueta = linea.get_etiqueta()
      diccionario[etiqueta] = contador
  return diccionario

def agregar_segundo(diccionario, lineas):
  contador = 16
  for linea in lineas:
    if linea.tipo == 'A' and not (linea.get_valor() in diccionario):
      if not linea.get_valor().isdigit():
        diccionario[linea.get_valor()] = contador
        contador = contador + 1
  return diccionario


# Main

In [11]:
def main(diccionario, programa, nombre, ruta):
  lineas = []
  programa = del_com_espacios(programa) #Limpiamos los comentarios y espacios en blanco
  for linea in programa.split("\n"):
    linea = parseo(linea)
    if(linea.tipo == None):
      continue
    lineas.append(linea)

  diccionario = agregar_al_diccionario(diccionario, lineas)
  diccionario = agregar_segundo(diccionario, lineas)

  with open(f"{ruta}/{nombre}.hack", "w") as archivo_hack:
    for i in lineas:
      instruccion = ""
      if(i.tipo == 'C'):
        instruccion = "111" + codigo_operacion(i.get_operacion()) + codigo_destino(i.get_destino()) + codigo_salto(i.get_salto())
        archivo_hack.write(instruccion + '\n')
      elif(i.tipo == 'A'):
        instruccion= codigo_valor(i.get_valor(), diccionario)
        instruccion = "0" +  instruccion
        archivo_hack.write(instruccion + '\n')

# Ensamblador

In [13]:
def ensamblador():
  #Si no lo han hecho, aqui podrían cargar sus programas
  add, max, maxL, pong, pongL, rect, rectL = cargarProgramas()

  diccionario = {
    "SCREEN" : 16384,
    "KBD" : 24576,
    "SP" : 0,
    "LCL" : 1,
    "ARG" : 2,
    "THIS" : 3,
    "THAT": 4
    }

  for i in range(16):
    llave = "R" + str(i)
    diccionario[llave] = i

#Traducimos cada uno de los programas por medio de la main:
  main(diccionario, add, 'Add', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, max, 'Max', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, maxL, 'MaxL', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, pong, 'Pong', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, pongL, 'PongL', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, rect, 'rect', '/content/drive/MyDrive/Arquitectura/Practica4')
  main(diccionario, rectL, 'rectL', '/content/drive/MyDrive/Arquitectura/Practica4')

# Traducir y almacenar los programas en el Drive por medio del Ensamblador

In [14]:
ensamblador()